1. Kompresja obrazów z użyciem sieci neuronowej  

·         Zbiór danych Fashion-MNIST  

·         Budujemy sieć neuronową dla kompresji danych, a następnie próbujemy sprawdzić czy sieć jest wstanie rozpoznać (poprawnie skwalifikować) dane z dekodera. Proszę również pokazać że sieć do kompresji można „rozciąć” na układ koder-dekoder. 

Tworzymy auto-enkoder i w pierwszej kolejności chcemy pokazać zależność błędu pomiędzy wyjściem a wejściem w stosunku do stopnia kompresji, 

Następnie patrzymy dla jakiś sensownych – wybranych - punktów z powyższej krzywej jak wygląda klasyfikacja danych zakodowanych.        Dla każdego wyniku stosujemy różne miary błędu (ewaluacji) MSE, macierz pomyłek, krzywe ROC,  specyficzność czułość, pole pod ROC itp.  

·         Wyniki proszę porównać z istniejącymi w literaturze.  

In [1]:
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop,Adam
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization

# loading dataset
(X, Y), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print("Train dataset shape:", X.shape, Y.shape)
print("Test dataset shape:", x_test.shape, y_test.shape)

Train dataset shape: (60000, 28, 28) (60000,)
Test dataset shape: (10000, 28, 28) (10000,)


# Uczenie modelu na oryginalnym zbiorze danych

*   Element listy
*   Element listy



In [2]:
# preprocessing data
X = X / 255.0
x_test = x_test / 255.0
train_size = X.shape[0]
test_size = x_test.shape[0]
img_shape = X.shape[1]

x_train_array = X.reshape(train_size, img_shape, img_shape, 1)
x_test_array = x_test.reshape(test_size, img_shape, img_shape, 1)

In [3]:
# preprocessing lables
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']
num_classes = len(class_names)
Y_ohe = tf.keras.utils.to_categorical(Y, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
# prepering for data for training
validation = 0.2
batch_size = 64
num_epochs = 5
patience = 10

x_train, x_val, y_train, y_val = train_test_split(x_train_array, Y_ohe, test_size=validation)

In [5]:
# building model
model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(28,28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dense(10, activation=tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               802944    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 804,554
Trainable params: 804,554
Non-trainable params: 0
__________________________________________________

In [6]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience)

model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

train_model = model.fit( x_train, y_train,
                  batch_size=batch_size,
                  epochs=num_epochs,
                  verbose=1,
                  validation_data=(x_val, y_val),
                  callbacks=[early_stop])

Epoch 1/5
750/750 [==============================] - 22s 29ms/step - loss: 0.4302 - accuracy: 0.8486 - val_loss: 0.3162 - val_accuracy: 0.8862
Epoch 2/5
750/750 [==============================] - 20s 27ms/step - loss: 0.2887 - accuracy: 0.8952 - val_loss: 0.2885 - val_accuracy: 0.8960
Epoch 3/5
750/750 [==============================] - 21s 28ms/step - loss: 0.2329 - accuracy: 0.9163 - val_loss: 0.2552 - val_accuracy: 0.9078
Epoch 4/5
750/750 [==============================] - 21s 28ms/step - loss: 0.1959 - accuracy: 0.9293 - val_loss: 0.2483 - val_accuracy: 0.9088
Epoch 5/5
750/750 [==============================] - 21s 27ms/step - loss: 0.1668 - accuracy: 0.9398 - val_loss: 0.2419 - val_accuracy: 0.9168


In [8]:
score = model.evaluate(x_test_array, y_test, steps=math.ceil(10000/32))
# checking the test loss and test accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.2655 - accuracy: 0.9066
Test loss: 0.26552459597587585
Test accuracy: 0.9065999984741211
